# Definir diretório raiz

In [1]:
from tkinter import filedialog
from tkinter import *
import os 
import pandas as pd
from PIL import Image

root = Tk()
#root.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
root.dir_raiz = filedialog.askdirectory()
print (root.dir_raiz)


os.chdir(root.dir_raiz)




C:/Users/joser/Documents/PROJETOS_DS_DBLab/Mask_RCNN


# DEFFINIÇÂO DE Nome dos Arquivos E Quantidades

In [2]:


#root.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
root.filename = filedialog.askdirectory()
print (root.filename)

imagens_produto_path = root.filename + "/" 
nome_produto = root.filename.split("/")
nome_produto = nome_produto[len(nome_produto)-1]


root.destroy()


C:/Users/joser/Documents/PROJETOS_DS_DBLab/Mask_RCNN/images/garrafa_1


In [3]:
#pip install natsort

# Analisar fotos do meu repositorio

In [4]:
from os import listdir
from os.path import isfile, join
from natsort import natsorted

arquivos_imagens = [f for f in natsorted(listdir(imagens_produto_path)) if isfile(join(imagens_produto_path, f))]

#print(arquivos_imagens)


# Prepare Model

In [5]:
import sys
import random
import math
import numpy as np

import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pycocotools

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
#import coco
from samples.coco import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = imagens_produto_path;

Using TensorFlow backend.


# Create Inference Object

In [6]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

In [7]:


# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Prediction and Visualization

# Cria diretorio com nome do produto na Pasta de OUTPUT das mascaras e coordenadas

In [9]:
path = "output\\"+nome_produto

try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)

Creation of the directory output\garrafa_1 failed


# Passa as imagens de produto da pasta para gerar coordenas e mascaras

In [18]:


coordenadas = np.empty([1, 5], dtype=int)

df_fileName = pd.DataFrame()
nomes_masks = pd.DataFrame()

#print(coordenadas.shape)

#for i in range(0,len(arquivos_imagens)-1):
    #print(len(arquivos_imagens))

for i in range(0, 1):
    
    path = imagens_produto_path + arquivos_imagens[i]
    image = skimage.io.imread(path)
    print(path)
    df_fileName[i] = ["images\\"+nome_produto+"\\"+arquivos_imagens[i]]
    # Run detection
    results = model.detect([image], verbose=0)
  
    # Visualize results
    r = results[0]

    b = np.empty([1, 5], dtype=int)
    #print('b 0 ',b)

    b[0,:-1] = r['rois'][0]
  
    

    b[0,4] = i

    coordenadas = np.append(coordenadas, b, axis=0)
  
    imagem_export = r['masks'][:,:,0]
    im = Image.fromarray(imagem_export.astype('uint8')*255)
    im.save("output\\"+nome_produto+"\\mask_("+"{}".format(i)+").png")
    nomes_masks[i] = ["output\\"+nome_produto+"\\mask_("+"{}".format(i)+").png"]

coordenadas = np.delete(coordenadas, 0, 0)

names = ["Ymin", "Xmin", "Ymax", "Xmax", "id"]
df = pd.DataFrame(coordenadas, columns=names)


df["name"] = df_fileName
df["mask"] = nomes_masks

df.to_csv("output\\"+nome_produto+"\\"+nome_produto+".csv", index=True, header=True, sep=',')

print(df)
print("done")


C:/Users/joser/Documents/PROJETOS_DS_DBLab/Mask_RCNN/images/garrafa_1/prod_(0).png
   Ymin  Xmin  Ymax  Xmax  id                           name  \
0   104   127   437   237   0  images\garrafa_1\prod_(0).png   

                            mask  
0  output\garrafa_1\mask_(0).png  
done


# Background Removal

In [ ]:


# Load a random image from the images folder
#for i in range(0,351):
for i in range(0,1):
    path = 'images/garrafa_2/garrafa2('+"{}".format(i)+').png'
    image = skimage.io.imread(path)
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
    results = model.detect([image], verbose=1)

In [ ]:
# Visualize results
r = results[0]

r['masks'].shape

#r['masks'].transpose(1, 0, 2).shape

image.shape

#print(r['masks'])

#np.savetxt("images/coord_mascara.csv", r['masks'], delimiter=",", fmt='%s')
#print(done)

#visualize.apply_mask(image[:,:,:2], r['masks'], [1, 0, 0], alpha=1.0)


visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                       class_names, r['scores'])

#visualize.save_image(image, "mascara_197", r['rois'], r['masks'], r['class_ids'], 
#                        r['scores'], class_names, mode = 3)

In [ ]:
#PARA TODAS AS IMAGENS GERAR MASCARAS

# Load a random image from the images folder
for i in range(0,n_aquivos):

    path = 'images/garrafa_2/garrafa2('+"{}".format(i)+').png'
    image = skimage.io.imread(path)
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
    results = model.detect([image], verbose=1)
    
    # Visualize results
    r = results[0]
    
  
    if results[0]['rois'].shape == (1,4):
    
        visualize.save_image(image, 'mask_('+"{}".format(i)+')', r['rois'], r['masks'], r['class_ids'], 
                        r['scores'], class_names, mode = 3)
        print(i)
     
print('done');
    
    